In [1]:
import multiprocessing as mp
import numpy as np
import pandas as pd
import skimage.io
import random
import copy
import math

In [2]:
def image_to_pixelmap(image_path: str) -> np.array:
    pixel_map = skimage.io.imread(image_path).copy()
    
    # Remove alpha component if it's a PNG file.
    if image_path[-3::] == "png":
        width, height, _ = pixel_map.shape
        pixel_map.resize((width, height, 3))
    return pixel_map

def color_distance(r1, g1, b1, r2, g2, b2):
    return math.sqrt(pow(float(r1) - r2, 2) + pow(float(g1) - g2, 2) + pow(float(b1) - b2, 2))

jpg_file_path = "./images/003.jpg"
png_file_path = "./images/002.png"

In [3]:
initial_pixels_map = image_to_pixelmap(jpg_file_path)
pixels_map = initial_pixels_map.copy()

# Compute clusters value

In [ ]:
def is_in_array(array, subarray):
    for i in range(len(array)):
        if subarray[0]==array[i][0] and subarray[1]==array[i][1] and subarray[2]==array[i][2]:
            return True
    return False
    

width, height, _ = pixels_map.shape

n_clusters = 8
clusters_points = [[0, 0, 0] for i in range(n_clusters)]
new_clusters_points = [[0, 0, 0] for i in range(n_clusters)]
nearest_cluster = copy.deepcopy(pixels_map)
nearest_cluster.resize((width, height, 1))


for i in range(n_clusters):
    x = random.randrange(0, width)
    y = random.randrange(0, height)
    while is_in_array(clusters_points, pixels_map[x][y]):
        x = random.randrange(0, width)
        y = random.randrange(0, height)
    clusters_points[i] = pixels_map[x][y]

print(clusters_points)

data = []

for x in range(width):
    for y in range(height):
        data.append((x, y, pixels_map[x][y]))
    

def compute_distances(data):
    distances = []
    x, y, color = data
    for i in range(8):
        distances.append(color_distance(*clusters_points[i], *color))
    return (x, y, distances.index(min(distances)))
    
for j in range(2):

    with mp.Pool(4) as p:
        result = p.map(compute_distances, data)
    
    print(result[0])
    break

    unique, counts = np.unique(nearest_cluster, return_counts=True)
    print(dict(zip(unique, counts)))
    
    for x in range(width):
        for y in range(height):
            cluster_index = nearest_cluster[x][y][0]
            new_clusters_points[cluster_index] = np.add(new_clusters_points[cluster_index], pixels_map[x][y])

    unique, counts = np.unique(nearest_cluster, return_counts=True)
    for index, count in list(zip(unique, counts)):
        new_clusters_points[index] = (new_clusters_points[index] / count).round()
    
    clusters_points = new_clusters_points
    new_clusters_points = [[0, 0, 0] for i in range(n_clusters)]

[array([36, 74, 33], dtype=uint8), array([222,  79,  62], dtype=uint8), array([34, 31, 16], dtype=uint8), array([125,   0, 105], dtype=uint8), array([238,  34,   0], dtype=uint8), array([255, 197, 124], dtype=uint8), array([63, 91, 14], dtype=uint8), array([38, 73,  7], dtype=uint8)]


# Transform pixel color

In [ ]:
for x in range(width):
    for y in range(height):
        pixels_map[x][y] = clusters_points[nearest_cluster[x][y][0]]

In [ ]:
print(pixels_map)

In [ ]:
skimage.io.imshow(initial_pixels_map)

In [ ]:
skimage.io.imshow(pixels_map)